In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 6)
import datetime as dt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np

from plotly.offline import iplot
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
file = 'docs/metall_rates.csv'
df = pd.read_csv(file, decimal=',')
df.info()

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'],
                           format = '%d.%m.%Y')
df.info()

In [ ]:
# df.tail(30)

In [ ]:
df.date = df.date.sort_values()

In [ ]:
# df.head(30)

In [ ]:
# df.tail(30)

In [ ]:
# create date_range
# end - 2020-10-17
yeardays = pd.date_range(start=pd.datetime(2020, 1, 1),
                           end=pd.datetime(2020, 10, 17))
yeardays

In [ ]:
# convert to DataFrame
yeardays = pd.DataFrame(yeardays)
yeardays = yeardays.rename(columns={0: "date"})
yeardays.info()

In [ ]:
# merge two df's 
ndf = pd.merge(left=yeardays,
               right=df,
               on="date",
              how='outer')
ndf.head()

In [ ]:
ndf = ndf.ffill(axis = 0)
ndf.head()

In [ ]:
df = ndf
df.info()

In [ ]:
# df['formatted_date'] = pd.to_datetime(df['datetime'])
df['month'] = df['date'].apply(lambda x: x.month)
df['week'] = df['date'].apply(lambda x: x.weekofyear)

In [ ]:
df['mean_per_week_gold'] = df.groupby('week')['gold'].transform('mean')
df['mean_per_week_silver'] = df.groupby('week')['silver'].transform('mean')
df['mean_per_month_gold'] = df.groupby('month')['gold'].transform('mean')
df['mean_per_month_silver'] = df.groupby('month')['silver'].transform('mean')
df.head(20)

In [ ]:
# def roll_days(datafr, metall, days):
#     datafr[f'rolling_{days}_{metall}'] = datafr[metall].rolling(window=days)

In [ ]:
df['rolling_7_gold'] = df['gold'].rolling(window=7).mean()
df['rolling_35_gold'] = df['gold'].rolling(window=35).mean()
df['rolling_50_gold'] = df['gold'].rolling(window=50).mean()
df.head(5)

In [ ]:
df['rolling_7_silver'] = df['silver'].rolling(window=7).mean()
df['rolling_35_silver'] = df['silver'].rolling(window=35).mean()
df['rolling_50_silver'] = df['silver'].rolling(window=50).mean()
df.head(5)

In [ ]:
df.index = df['date']
df.drop('date',axis = 1, inplace = True)
df.head()

In [ ]:
df.columns

In [ ]:
for i in df.columns:
    print(i)

In [ ]:
# silver_list = ['silver', 'mean_per_week_silver', 'mean_per_month_silver']
silver_list = ['silver', 
#                'mean_per_week_silver', 
#                'rolling_7_silver',
#                'rolling_35_silver',
               'rolling_50_silver']
# gold_list = ['gold', 'mean_per_week_gold', 'mean_per_month_gold']
gold_list = ['gold',
#              'mean_per_week_gold', 
#              'rolling_7_gold',
#              'rolling_35_gold',
             'rolling_50_gold']

In [ ]:
# iplot([{
#     'x': df.index,
#     'y': df[col],
#     'name': col
# }  for col in df.columns])

In [ ]:
# data = [{
#     'x': df.index,
#     'y': df[i],
#     'name': i
# }  for i in silver_list]

# iplot(data)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.gold, name='Цена на золото'))
fig.add_trace(go.Scatter(x=df.index, y=df.rolling_50_gold, name='Скользящая средняя за 50 дней'))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.silver, name='Цена на серебро'))
fig.add_trace(go.Scatter(x=df.index, y=df.rolling_50_silver, name='Скользящая средняя за 50 дней'))
fig.show()

In [ ]:
# data = [{
#     'x': df.index,
#     'y': df[i],
#     'name': i
# }  for i in gold_list] 

# iplot(data)

In [ ]:
hour, direction = np.meshgrid(np.arange(24), np.arange(1,3))
df = pd.DataFrame({"hour": hour.flatten(), "direction": direction.flatten()})
df["hourly_avg_count"] = np.random.randint(14,30, size=len(df))

plt.figure(figsize=(12,8))
sns.tsplot(df, time='hour', unit = "direction", 
               condition='direction', value='hourly_avg_count')

plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.title('График цен на серебро за 2020 год (цена дневная и недельная средняя)')

ax1 = df.rsti_price.plot(color='blue', grid=True, label='RSTI')
ax2 = df.brent_price.plot(color='red', grid=True, secondary_y=True, label='Brent')

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()

plt.legend(h1+h2, l1+l2, loc=1)
plt.show()

In [ ]:
def plot_for_list(metall_list, tit):
    for i in metall_list:
        df[i].plot(figsize=(20,6),
                   title=str(tit))

In [ ]:
plot_for_list(silver_list, 'График цен на серебро за 2020 год (цена дневная и недельная средняя)')

In [ ]:
plot_for_list(gold_list, 'График цен на золото за 2020 год (цена дневная и недельная средняя)')

In [ ]:
def seasonal_decompose_plot(result, i):
    fig, axes = plt.subplots(ncols = 1, nrows = 4, sharex = True, figsize = (12,10))

    # ----------------------------------------------------------------------------------------------------
    # plot the data
    # using this cool thread:
    # https://stackoverflow.com/questions/45184055/how-to-plot-multiple-seasonal-decompose-plots-in-one-figure
    # This allows us to have more control over the plots

    # plot the original data
    result.observed.plot(ax = axes[0], legend = False)
    axes[0].set_ylabel('График')
    axes[0].set_title(f"Декомпозиция временного ряда 2020 ({i})")

    # plot the trend
    result.trend.plot(ax = axes[1], legend = False)
    axes[1].set_ylabel('Тренд')

    # plot the seasonal part
    result.seasonal.plot(ax = axes[2], legend = False)
    axes[2].set_ylabel('Сезонная компонента')

    # plot the residual
    result.resid.plot(ax = axes[3], legend = False)
    axes[3].set_ylabel('Остаточная компонента')

    # ----------------------------------------------------------------------------------------------------
    # prettify the plot

    # get the xticks and the xticks labels
    # xtick_location = df.index.tolist()[::6]
    # xtick_labels = df["new_date"].tolist()[::6]

    # set the xticks to be every 6'th entry
    # every 6 months
    # ax.set_xticks(xtick_location)

    # chage the label from '1949-01-01 00:00:00' to this 'Jan-1949'
    # ax.set_xticklabels(xtick_labels, rotation=90, fontdict={'horizontalalignment': 'center', 'verticalalignment': 'center_baseline'});

In [ ]:
result = seasonal_decompose(df['gold'], model='multiplicative')
seasonal_decompose_plot(result, 'золото')

In [ ]:
result = seasonal_decompose(df['silver'], model='multiplicative')
seasonal_decompose_plot(result, 'серебро')

In [ ]:
# тут построить разложения по месяцам

In [ ]:
last_month = pd.to_datetime("2020-10-01").date()

In [ ]:
df.tail()

In [ ]:
df_last_month = df[last_month:]
df_last_month.tail()

In [ ]:
result = seasonal_decompose(df_last_month['silver'], model='multiplicative')
seasonal_decompose_plot(result, 'серебро')

In [ ]:
result = seasonal_decompose(df_last_month['gold'], model='multiplicative')
seasonal_decompose_plot(result, 'золото')

In [ ]:
# золото и серебро

# x = df['date']
y1 = df['gold']
y2 = df['silver']

# Plot Line1 (Left Y Axis)
fig, ax1 = plt.subplots(1,1,figsize=(16,9), dpi= 80)
# ax1.plot(x, y1, color='tab:red')
ax1.plot(y1, color='tab:red')

# Plot Line2 (Right Y Axis)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
# ax2.plot(x, y2, color='tab:blue')
ax2.plot(y2, color='tab:blue')

# Decorations
# ax1 (left Y axis)
ax1.set_xlabel('Месяцы', fontsize=20)
# ax1.tick_params(axis='x', rotation=0, labelsize=12)
ax1.set_ylabel('Цены на золото', color='tab:red', fontsize=20)
ax1.tick_params(axis='y', rotation=0, labelcolor='tab:red' )
ax1.grid(alpha=.4)

# ax2 (right Y axis)
ax2.set_ylabel("Цены на серебро", color='tab:blue', fontsize=20)
ax2.tick_params(axis='y', labelcolor='tab:blue')
# ax2.set_xticks(np.arange(0, len(x), 60))
# ax2.set_xticklabels(x[::60], rotation=90, fontdict={'fontsize':10})
ax2.set_title("Сравнение динамики цен на золото и серебро", fontsize=22)
fig.tight_layout()
plt.show()

In [ ]:
# тут можно построить такие сравнения по месяцам

In [ ]:
# график сезонов
def metall_trend_monthly(metall, tit):
    mycolors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange', 'tab:brown', 'tab:grey', 'tab:pink', 'tab:olive', 'deeppink', 'steelblue']
    months = ['январь','февраль','март','апрель','май','июнь','июль','август','сентябрь','октябрь']
    plt.figure(figsize=(16,10), dpi= 80)
    plt.suptitle('Сравнение месячных трендов за 2020 год', fontsize=15)
    plt.title(str(tit))
    plt.xlabel('День месяца', fontsize=14)
    plt.ylabel('Цена', fontsize=14)
    lines = []
    for i, y in enumerate(df[metall].groupby(pd.Grouper(freq='M'))):
        p,=plt.plot(range(len(y[1])),y[1], color=mycolors[i], label=months[i])
        lines.append(p)
    plt.legend(handles=lines)

In [ ]:
metall_trend_monthly('silver', 'Серебро')

In [ ]:
metall_trend_monthly('gold', 'Золото')

In [ ]:
# по неделям - работает плохо

# mycolors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange', 'tab:brown', 'tab:grey', 'tab:pink', 'tab:olive', 'deeppink', 'steelblue']
# week_days = ['понедельник','вторник','среда','четверг','пятница','суббота','воскресенье']
# plt.figure(figsize=(16,10), dpi= 80)
# plt.title('Сравнение сезонных трендов по неделям')
# plt.xlabel('day of week', fontsize=14)
# plt.ylabel('price', fontsize=14)
# lines = []
# for i, y in enumerate(df['silver'].groupby(pd.Grouper(freq='W'))):
#     p,=plt.plot(range(len(y[1])),y[1], color=mycolors[i], label=week_days[i])
#     lines.append(p)
# plt.legend(handles=lines)

In [ ]:
# from datetime import date

In [ ]:
allbanks = pd.read_csv('docs/allbanks.csv', 
                       encoding='cp1251', 
                       sep=';', 
                       index_col=0)
allcoins = pd.read_csv('docs/allcoins.csv', 
                       encoding='cp1251', 
                       sep=';', 
                       index_col=0)
prices = pd.read_csv('docs/prices.csv', 
                     encoding='cp1251', 
                     sep=';')

In [ ]:
allbanks.head()

In [ ]:
allcoins.head()

In [ ]:
allcoins.DT = pd.to_datetime(allcoins.DT, unit='s').dt.date
allcoins.head()

In [ ]:
prices.head()

In [ ]:
prices.date = pd.to_datetime(prices.date, unit='s').dt.date
prices.head()

In [ ]:
result_df = prices.merge(allcoins, left_on = 'id_coin', right_index=True)
result_df.head()

In [ ]:
result_df = result_df.merge(allbanks, left_on = 'id_bank', right_index=True)
result_df.head()

In [ ]:
# visualization

In [ ]:
allcoins['metal'].unique()

In [ ]:
nedrag_list = ['Медно-никелевый сплав',
               'Сталь с латунным/никелевым гальваническим покрытием',
               'Сталь с латунным гальваническим покрытием', 
               'Латунь/Мельхиор',
               'Сталь с никелевым гальваническим покрытием', 
               'Мельхиор',
               'Нейзильбер', 
               'Медь, никель', 
               'Латунь',
               'Латунь/Медь, никель', 
               'Нейзильбер/Латунь',
               'Медь, цинк/Медь, никель']

In [ ]:
gold_list = ['Au-999',
             'Au-900']

In [ ]:
silver_list = ['Ag-925', 
               'Ag-999', 
               'Ag-900', 
               'Ag-500']

In [ ]:
combine_metal_list = ['Ag-900/Au-900', 
                      'Ag-925/Au-999', 
                      'Ag-925/позолота', 
                      'Pd-999', 
                      'Pt-999', 
                      'AU-900/Ag-925', 
                      'Ag-925/Au-999', 
                      'Ag-900/Au-900', 
                      'Au-900/Ag-900']

In [ ]:
allcoins_gold = allcoins[allcoins['metal'].isin(gold_list)]
allcoins_gold.metal.unique()

In [ ]:
allcoins_silver = allcoins[allcoins['metal'].isin(silver_list)]
allcoins_silver.metal.unique()

In [ ]:
allcoins_nedrag = allcoins[allcoins['metal'].isin(nedrag_list)]
allcoins_nedrag.metal.unique()

In [ ]:
allcoins_combine = allcoins[allcoins['metal'].isin(combine_metal_list)]
allcoins_combine.metal.unique()

In [ ]:
startdate = pd.to_datetime("2017-01-01").date()

In [ ]:
allcoins_gold_3_years = allcoins_gold[allcoins_gold['DT'] >= startdate]
allcoins_gold_3_years.head()

In [ ]:
allcoins_silver_3_years = allcoins_silver[allcoins_silver['DT'] >= startdate]
allcoins_silver_3_years.head()

In [ ]:
allcoins_nedrag_3_years = allcoins_nedrag[allcoins_nedrag['DT'] >= startdate]
allcoins_nedrag_3_years.head()

In [ ]:
allcoins_combine_3_years = allcoins_combine[allcoins_combine['DT'] >= startdate]
allcoins_combine_3_years.head()

In [ ]:
def get_plot_count(datafr, i):
    plt.figure(figsize=(10,5))
    ax = sns.countplot(datafr['Nominal'])
    ax.set(xlabel='Номинал монет', 
           ylabel='Количество видов')
    plt.suptitle('Количество видов монет за последние 3 года', size = 14)
#     plt.figsize=(20, 20)
    plt.title(str(i), size = 12)

    plt.show()

In [ ]:
get_plot_count(allcoins_gold_3_years, 'золото')

In [ ]:
get_plot_count(allcoins_silver_3_years, '(серебро)')

In [ ]:
get_plot_count(allcoins_nedrag_3_years, '(недрагоценные металлы)')

In [ ]:
# ВОТ ОТСЮДА

In [ ]:
result_df.head()

In [ ]:
result_df_stat = result_df[['sell_price','Nominal']]
result_df_stat.groupby('Nominal').describe()

In [ ]:
# .groupby('Nominal').describe()

In [ ]:
result_df_gold = result_df[result_df['metal'].isin(gold_list)]
result_df_gold.Nominal.unique()

In [ ]:
result_df_silver = result_df[result_df['metal'].isin(silver_list)]

In [ ]:
result_df_nedrag = result_df[result_df['metal'].isin(nedrag_list)]

In [ ]:
# describe

In [ ]:
result_df_gold[['sell_price','Nominal']].groupby('Nominal').describe()

In [ ]:
result_df_silver[['sell_price','Nominal']].groupby('Nominal').describe()

In [ ]:
result_df_nedrag[['sell_price','Nominal']].groupby('Nominal').describe()

In [ ]:
result_df_nedrag[['sell_price','Nominal']].groupby('Nominal').describe()

In [ ]:
result_df_nedrag[['sell_price','coin_name', 'rus_name']].groupby('sell_price').describe()

In [ ]:
result_df_gold.info()

In [ ]:
result_df_gold.Nominal.unique()

In [ ]:
for i in set(result_df_gold.Nominal.unique()):
    hidden = result_df_gold
    hidden['Nominal'] = result_df_gold[result_df_gold['Nominal']==str(i)]
    hidden.groupby('cat_number').describe()

In [ ]:
# result_df_gold[['sell_price','Nominal']].head()

In [ ]:
# res_clear = result_df_gold[['sell_price','Nominal']]
# res_clear = res_clear[res_clear['sell_price']>30000] #мусорные цены, надо вычистить в базе
# nominal_list = set(res_clear['Nominal'])

In [ ]:
# nominal_list

In [ ]:
# for i in nominal_list:
#     hidden_df = res_clear[res_clear['Nominal']==i]
#     sns.histplot(hidden_df['sell_price'], 
#                 color="gold",
#                 bins = 10).set(title=i)
#     plt.show()

In [ ]:
# res_clear = result_df_silver[['sell_price','Nominal']]
# res_clear = res_clear[res_clear['sell_price']>2000] #мусорные цены, надо вычистить в базе
# nominal_list = set(res_clear['Nominal'])

In [ ]:
# for i in nominal_list:
#     hidden_df = res_clear[res_clear['Nominal']==i]
#     sns.histplot(hidden_df['sell_price'], 
#                 color="silver",
#                 bins=100).set(title=i)
#     plt.show()

In [ ]:
# df.to_csv('docs/drags.csv')